In [1]:
import pandas as pd
from helm.benchmark.runner import InstanceGenerations,GenerationSummary
from typing import Any, List
import json
from helm.common.request import (GeneratedOutput, Token)

import PostMetric
import pandas as pd

from helm.benchmark.metrics.statistic import Stat
from typing import Dict, Optional

from helm.benchmark.augmentations.perturbation_description import (
    PerturbationDescription)
from dataclasses import dataclass
import os
import re

/opt/miniconda3/envs/crfm-helm2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch version 2.6.0 available.


In [2]:
from dataclasses import dataclass
from helm.common.request import (GeneratedOutput, Token)

from helm.benchmark.runner import InstanceGenerations,GenerationSummary
from typing import List
from process_gens import get_completion_from_examples
@dataclass(frozen=False)
class FairseqOutput:
    sentence_id:int=None
    prompt:str=None
    reference:str=None
    generated_output:GeneratedOutput=None

In [3]:


# fairseq_output=f"/Users/lukecadigan/thesis/fairseq/{activation}_generate_results.txt"
# fairseq_output=f"/Users/lukecadigan/thesis/helm/{activation}_results_mini.txt"
# '/Users/lukecadigan/thesis/fairseq/{activation}_results/seed_0/generate-test.txt'
# fairseq_output="/Users/lukecadigan/thesis/fairseq/results/iwslt/{activation}/"

def get_fairset_output_from_lines(lines) -> FairseqOutput:
    fairseq_output= FairseqOutput()
    for i in range(0,num_lines, step_size):
        logprobs=None
        words=None
        sentence_prob=None
        sentence_text=None
        
        for raw_line in lines:
            line=raw_line.strip()
            split_on_tab = line.split("\t")


            
            id=split_on_tab[0]
            split_id=id.split("-")
            indicator=split_id[0]
            sentence_id=split_id[1]        
            content=split_on_tab[-1]
            content_prob = split_on_tab[1]if len(split_on_tab)==3 else None

            #sanity check
            if indicator in ['H', 'D']:
                assert len(split_on_tab)==3
            else: 
                assert len(split_on_tab)==2
            # if indicator not in results_dict[sentence_id]:
            if indicator == "S":
                fairseq_output.prompt=content
                fairseq_output.sentence_id=sentence_id
            elif indicator == "T":
                fairseq_output.reference=content
            elif indicator == "H":
                words=content.split(" ")+["<eos>"]
                sentence_prob=content_prob
            elif indicator == "D":
                sentence_text=content
            elif indicator == "P":
                logprobs=content.split(" ")
        tokens=[]    
        for logprob, text in zip(logprobs, words):
            tokens.append(Token(text=text,logprob=logprob))
        fairseq_output.generated_output = GeneratedOutput(text=sentence_text, logprob=sentence_prob, tokens=tokens)
    return fairseq_output


def fairseq_outputs_to_instance_generations(fairseq_output_list: List[FairseqOutput]) -> InstanceGenerations:
    assert len(fairseq_output_list)>0
    first_fairseq_output= fairseq_output_list[0]

    examples=[fairseq_output.generated_output for fairseq_output in fairseq_output_list]
    examples, completion, completion_logprob=get_completion_from_examples(examples)
    return InstanceGenerations(instance_id=first_fairseq_output.sentence_id, 
                        prompt=first_fairseq_output.prompt, 
                        reference=first_fairseq_output.reference, 
                        completion=completion,
                        completion_logprob=completion_logprob,
                        full_prompt=first_fairseq_output.prompt,
                        examples=examples 
                        )



fairseq_output_by_id={}
num_outputs=1000


#ensure that there are 2 outputs
#ensure that there ids are 1749 and 3668

activation="softmax"
for seed in range(100):
    print(f"Processing seed {seed}")
    fairseq_file=f"/Users/lukecadigan/thesis/fairseq/results/iwslt/{activation}/seed_{seed}/generate-test.txt"
    step_size=5

    
    with open(fairseq_file, 'r') as file:    
        all_lines=file.readlines()
        len_all_lines=len(all_lines)
        num_lines=min(num_outputs*step_size, len_all_lines)
        # num_lines=len_all_lines
        for i in range(0, num_lines, step_size):
            
            lines=all_lines[i:i+step_size]
            fairseq_output :FairseqOutput= get_fairset_output_from_lines(lines)

            if fairseq_output.sentence_id not in fairseq_output_by_id:
                fairseq_output_by_id[fairseq_output.sentence_id]=[]
            fairseq_output_by_id[fairseq_output.sentence_id].append(fairseq_output)
instance_generations = [fairseq_outputs_to_instance_generations(fairseq_outputs) for fairseq_outputs in fairseq_output_by_id.values()]

print(f"Num instance generations is {len(instance_generations)}")
generation_summary=GenerationSummary(task_name="fairseq_wmt",instance_generations=instance_generations, adapter_spec=None)



Processing seed 0
Processing seed 1
Processing seed 2
Processing seed 3
Processing seed 4
Processing seed 5
Processing seed 6
Processing seed 7
Processing seed 8
Processing seed 9
Processing seed 10
Processing seed 11
Processing seed 12
Processing seed 13
Processing seed 14
Processing seed 15
Processing seed 16
Processing seed 17
Processing seed 18
Processing seed 19
Processing seed 20
Processing seed 21
Processing seed 22
Processing seed 23
Processing seed 24
Processing seed 25
Processing seed 26
Processing seed 27
Processing seed 28
Processing seed 29
Processing seed 30
Processing seed 31
Processing seed 32
Processing seed 33
Processing seed 34
Processing seed 35
Processing seed 36
Processing seed 37
Processing seed 38
Processing seed 39
Processing seed 40
Processing seed 41
Processing seed 42
Processing seed 43
Processing seed 44
Processing seed 45
Processing seed 46
Processing seed 47
Processing seed 48
Processing seed 49
Processing seed 50
Processing seed 51
Processing seed 52
Pro

In [4]:
from helm.common.general import write, asdict_without_nones

# activation="sparsemax"
output_file_name=f"snellius_copies/helm_output/fairseq/wmt_14_language_pair_de_en_/fairseq_{activation}/1_beams/runs/fairseq/generation_summary.json"


write(
    output_file_name,
    json.dumps(asdict_without_nones(generation_summary),indent=2)
)

Writing 90529871 characters to snellius_copies/helm_output/fairseq/wmt_14_language_pair_de_en_/fairseq_softmax/1_beams/runs/fairseq/generation_summary.json


In [5]:
print(f"""

      
LOCAL_FILE={output_file_name}
REMOTE_DIR_EXT=helm/helm_output/fairseq/wmt_14_language_pair_de_en_/fairseq_{activation}/1_beams/runs/fairseq
echo scp $LOCAL_FILE lcadigan@snellius.surf.nl:~/$REMOTE_DIR_EXT
scp $LOCAL_FILE lcadigan@snellius.surf.nl:~/$REMOTE_DIR_EXT
""")








      
LOCAL_FILE=snellius_copies/helm_output/fairseq/wmt_14_language_pair_de_en_/fairseq_softmax/1_beams/runs/fairseq/generation_summary.json
REMOTE_DIR_EXT=helm/helm_output/fairseq/wmt_14_language_pair_de_en_/fairseq_softmax/1_beams/runs/fairseq
echo scp $LOCAL_FILE lcadigan@snellius.surf.nl:~/$REMOTE_DIR_EXT
scp $LOCAL_FILE lcadigan@snellius.surf.nl:~/$REMOTE_DIR_EXT

